In [21]:
from ktms import *
from MAInit import *
from MACalc import *
import re

In [22]:
from pymongo import MongoClient
client = MongoClient()

In [23]:
client = MongoClient('localhost', 27017)

In [24]:
db = client.adsE_database
collection = db.adsE_collection

In [25]:
def makejson(file):
    atoms = query(file, 'spacom')

    dic = {}

    res = re.match('(.*u\d).*', file)
    barefile = res.group(1) + '.traj'
    bareatoms = query(barefile, 'spacom')
    print(bareatoms)

    COatoms = query('CO.traj', 'scpacom')
    COene = COatoms.get_potential_energy()

    res = re.match('(\w.*)_(\d{3}).*', file)
    ele = res.group(1)
    mir = res.group(2)
    
    num = 0
    ads = 0
    res = re.match('.*n(\d+)_', file)
    if res:
        num = int(res.group(1))
        
    res = re.match('.*u(\d+).*', file)
    unit = int(res.group(1))

    res = re.match('.*no.*_(\w.)_n', file)
    if res:
        ads = res.group(1)

    x = atoms.cell[0][0]
    y = atoms.cell[1][1]
    area = x*y

    tot = 0
    for atom in atoms:
        if atom.tag == 1:
            tot += 1

    ene = atoms.get_potential_energy()
    bareene = bareatoms.get_potential_energy()
    totaladsene = ene - (COene*num + bareene)
    
    dic['name'] = file
    dic['element'] = ele
    dic['mirror'] = mir
    dic['adsorbate'] = ads
    dic['unitlength'] = unit
    dic['surfatomnum'] = tot
    dic['area'] = area
    dic['Energy'] = ene
    dic['totaladsEnergy'] = totaladsene
    dic['aveadsEne/suratom'] = totaladsene/tot
    dic['numberofads'] = num
    dic['ML'] = num/tot
    dic['density'] = num/area
    if num:
        dic['aveadsEne/ads'] = totaladsene/num
    else:
        dic['aveadsEne/ads'] = None
            
    for key in dic.keys():
        print('{}: {}'.format(key, dic[key]))
    
    return dic

In [26]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')
for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            print(i, files[i])

1 Ag_100_u1.traj
3 Ag_100_u1_no01_CO_n1_d3.traj
5 Ag_100_u1_no02_CO_n1_d3.traj
7 Ag_100_u1_no03_CO_n1_d3.traj
9 Ag_100_u2.traj
11 Ag_100_u2_no01_CO_n1_d9.traj
13 Ag_100_u2_no02_CO_n2_d3.traj
15 Ag_100_u2_no03_CO_n3_d3.traj
17 Ag_100_u2_no04_CO_n4_d3.traj
19 Ag_100_u2_no05_CO_n3_d3.traj
21 Ag_100_u2_no06_CO_n4_d3.traj
23 Ag_100_u2_no07_CO_n2_d6.traj
25 Ag_100_u2_no08_CO_n2_d4.traj
27 Ag_100_u2_no09_CO_n3_d3.traj
29 Ag_100_u2_no10_CO_n1_d9.traj
31 Ag_100_u2_no11_CO_n2_d3.traj
33 Ag_100_u2_no12_CO_n3_d3.traj
35 Ag_100_u2_no13_CO_n4_d3.traj
37 Ag_100_u2_no14_CO_n3_d3.traj
39 Ag_100_u2_no15_CO_n4_d3.traj
41 Ag_100_u2_no16_CO_n2_d6.traj
43 Ag_100_u2_no17_CO_n2_d4.traj
45 Ag_100_u2_no18_CO_n3_d3.traj
47 Ag_100_u2_no19_CO_n2_d3.traj
49 Ag_100_u2_no20_CO_n1_d9.traj
51 Ag_100_u2_no21_CO_n2_d3.traj
53 Ag_100_u2_no22_CO_n3_d3.traj
55 Ag_100_u2_no23_CO_n4_d3.traj
57 Ag_100_u2_no24_CO_n2_d6.traj
59 Ag_100_u3.traj
61 Ag_100_u3_no01_CO_n1_d15.traj
63 Ag_100_u3_no02_CO_n1_d15.traj
65 Ag_100_u3_no03_CO_

In [27]:
makejson(files[155])

Atoms(symbols='CoPt3CoPt3', pbc=[True, True, False], cell=[3.88603895417, 3.88603895417, 25.829058431255], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3]), calculator=SinglePointCalculator(...))
name: CoPt3_100_u1.traj
element: CoPt3
mirror: 100
adsorbate: 0
unitlength: 1
surfatomnum: 2
area: 15.101298753326667
Energy: -42.04234066
totaladsEnergy: 0.0
aveadsEne/suratom: 0.0
numberofads: 0
ML: 0.0
density: 0.0
aveadsEne/ads: None


{'name': 'CoPt3_100_u1.traj',
 'element': 'CoPt3',
 'mirror': '100',
 'adsorbate': 0,
 'unitlength': 1,
 'surfatomnum': 2,
 'area': 15.101298753326667,
 'Energy': -42.04234066,
 'totaladsEnergy': 0.0,
 'aveadsEne/suratom': 0.0,
 'numberofads': 0,
 'ML': 0.0,
 'density': 0.0,
 'aveadsEne/ads': None}

In [28]:
env = 'local'
files = os.listdir('/home/katsuyut/database/')

for i in range(len(files)):
    if '.traj' in files[i]:
        if 'all' not in files[i]:
            if 'CO.traj' not in files[i]:
                print(i, files[i])

                post_data = makejson(files[i])
                result = collection.insert_one(post_data)
                print('One post: {0}'.format(result.inserted_id))

1 Ag_100_u1.traj
Atoms(symbols='Ag4', pbc=[True, True, False], cell=[2.979087029472652, 2.979087029472652, 26.319597920855003], tags=..., constraint=FixAtoms(indices=[0, 1]), calculator=SinglePointCalculator(...))
name: Ag_100_u1.traj
element: Ag
mirror: 100
adsorbate: 0
unitlength: 1
surfatomnum: 1
area: 8.874959529172191
Energy: -8.12811404
totaladsEnergy: 0.0
aveadsEne/suratom: 0.0
numberofads: 0
ML: 0.0
density: 0.0
aveadsEne/ads: None
One post: 5d5ea176bfa5790697e5a2f2
3 Ag_100_u1_no01_CO_n1_d3.traj
Atoms(symbols='Ag4', pbc=[True, True, False], cell=[2.979087029472652, 2.979087029472652, 26.319597920855003], tags=..., constraint=FixAtoms(indices=[0, 1]), calculator=SinglePointCalculator(...))
name: Ag_100_u1_no01_CO_n1_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 1
surfatomnum: 1
area: 8.874959529172191
Energy: -22.44298044
totaladsEnergy: 0.1705699100000011
aveadsEne/suratom: 0.1705699100000011
numberofads: 1
ML: 1.0
density: 0.11267657015369789
aveadsEne/ads: 0.1705

Atoms(symbols='Ag16', pbc=[True, True, False], cell=[5.958174058945304, 5.958174058945304, 26.319597920855003], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Ag_100_u2_no12_CO_n3_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -74.92459206
totaladsEnergy: 0.9913197200000212
aveadsEne/suratom: 0.2478299300000053
numberofads: 3
ML: 0.75
density: 0.08450742761527343
aveadsEne/ads: 0.3304399066666737
One post: 5d5ea177bfa5790697e5a302
35 Ag_100_u2_no13_CO_n4_d3.traj
Atoms(symbols='Ag16', pbc=[True, True, False], cell=[5.958174058945304, 5.958174058945304, 26.319597920855003], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Ag_100_u2_no13_CO_n4_d3.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 35.499838116688764
Energy: -88.80871776
totaladsEnergy: 1.5926303300000058
aveadsEne/

Atoms(symbols='Ag64', pbc=[True, True, False], cell=[11.916348117890609, 11.916348117890609, 26.319597920855003], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]), calculator=SinglePointCalculator(...))
name: Ag_100_u4_no01_CO_n1_d21.traj
element: Ag
mirror: 100
adsorbate: CO
unitlength: 4
surfatomnum: 16
area: 141.99935246675506
Energy: -144.48860057
totaladsEnergy: -0.10997062999996388
aveadsEne/suratom: -0.006873164374997742
numberofads: 1
ML: 0.0625
density: 0.007042285634606118
aveadsEne/ads: -0.10997062999996388
One post: 5d5ea177bfa5790697e5a314
71 Ag_100_u4_no02_CO_n1_d21.traj
Atoms(symbols='Ag64', pbc=[True, True, False], cell=[11.916348117890609, 11.916348117890609, 26.319597920855003], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]), calculator=SinglePointCalc

One post: 5d5ea177bfa5790697e5a325
105 Ag_111_u2_no10_CO_n3_d3.traj
Atoms(symbols='Ag16', pbc=[True, True, False], cell=[[5.958174058945304, 0.0, 0.0], [2.979087029472652, 5.159930095216073, 0.0], [0.0, 0.0, 27.29724312155167]], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Ag_111_u2_no10_CO_n3_d3.traj
element: Ag
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 30.743761639287584
Energy: -75.16671383
totaladsEnergy: 1.2113686599999909
aveadsEne/suratom: 0.3028421649999977
numberofads: 3
ML: 0.75
density: 0.09758077216440188
aveadsEne/ads: 0.4037895533333303
One post: 5d5ea177bfa5790697e5a326
107 Ag_111_u2_no11_CO_n4_d3.traj
Atoms(symbols='Ag16', pbc=[True, True, False], cell=[[5.958174058945304, 0.0, 0.0], [2.979087029472652, 5.159930095216073, 0.0], [0.0, 0.0, 27.29724312155167]], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Ag_111_u2_no11_CO_n4

name: Ag_111_u3.traj
element: Ag
mirror: 111
adsorbate: 0
unitlength: 3
surfatomnum: 9
area: 69.17346368839706
Energy: -75.05788028
totaladsEnergy: 0.0
aveadsEne/suratom: 0.0
numberofads: 0
ML: 0.0
density: 0.0
aveadsEne/ads: None
One post: 5d5ea177bfa5790697e5a334
135 Ag_111_u3_no01_CO_n1_d15.traj
Atoms(symbols='Ag36', pbc=[True, True, False], cell=[[8.937261088417957, 0.0, 0.0], [4.468630544208978, 7.7398951428241105, 0.0], [0.0, 0.0, 27.29724312155167]], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]), calculator=SinglePointCalculator(...))
name: Ag_111_u3_no01_CO_n1_d15.traj
element: Ag
mirror: 111
adsorbate: CO
unitlength: 3
surfatomnum: 9
area: 69.17346368839706
Energy: -89.49222777
totaladsEnergy: 0.05108882000000392
aveadsEne/suratom: 0.005676535555555991
numberofads: 1
ML: 0.1111111111111111
density: 0.014456410691022499
aveadsEne/ads: 0.05108882000000392
One post: 5d5ea177bfa5790697e5a335
137 Ag_111_u3_no02_CO_n1_d15.traj


...

483 Pt_100_u2_no011_CO_n5_d1.traj
Atoms(symbols='Pt16', pbc=[True, True, False], cell=[5.643844235708376, 5.643844235708376, 25.986200796545], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Pt_100_u2_no011_CO_n5_d1.traj
element: Pt
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 31.852977756938664
Energy: -154.21517803
totaladsEnergy: -0.8397991200000092
aveadsEne/suratom: -0.2099497800000023
numberofads: 5
ML: 1.25
density: 0.15697119553950745
aveadsEne/ads: -0.16795982400000184
One post: 5d5ea17abfa5790697e5a3e2
485 Pt_100_u2_no012_CO_n5_d1.traj
Atoms(symbols='Pt16', pbc=[True, True, False], cell=[5.643844235708376, 5.643844235708376, 25.986200796545], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Pt_100_u2_no012_CO_n5_d1.traj
element: Pt
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 31.852977756938664
Energy: -153.64112679
totalad

Atoms(symbols='Pt16', pbc=[True, True, False], cell=[5.643844235708376, 5.643844235708376, 25.986200796545], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Pt_100_u2_no06_CO_n4_d3.traj
element: Pt
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 31.852977756938664
Energy: -143.30580451
totaladsEnergy: -4.415861910000018
aveadsEne/suratom: -1.1039654775000045
numberofads: 4
ML: 1.0
density: 0.12557695643160596
aveadsEne/ads: -1.1039654775000045
One post: 5d5ea17abfa5790697e5a3f1
515 Pt_100_u2_no07_CO_n2_d5.traj
Atoms(symbols='Pt16', pbc=[True, True, False], cell=[5.643844235708376, 5.643844235708376, 25.986200796545], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Pt_100_u2_no07_CO_n2_d5.traj
element: Pt
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 31.852977756938664
Energy: -113.10810331
totaladsEnergy: -3.189033330000015
aveadsEne/su

Atoms(symbols='Pt16', pbc=[True, True, False], cell=[5.643844235708376, 5.643844235708376, 25.986200796545], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Pt_100_u2_no20_CO_n1_d9.traj
element: Pt
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 31.852977756938664
Energy: -96.54722712
totaladsEnergy: -1.1135934500000104
aveadsEne/suratom: -0.2783983625000026
numberofads: 1
ML: 0.25
density: 0.03139423910790149
aveadsEne/ads: -1.1135934500000104
One post: 5d5ea17abfa5790697e5a3ff
543 Pt_100_u2_no21_CO_n2_d3.traj
Atoms(symbols='Pt16', pbc=[True, True, False], cell=[5.643844235708376, 5.643844235708376, 25.986200796545], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Pt_100_u2_no21_CO_n2_d3.traj
element: Pt
mirror: 100
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 31.852977756938664
Energy: -111.41059467
totaladsEnergy: -1.4915246900000056
aveadsEne/

Atoms(symbols='Pt4', pbc=[True, True, False], cell=[[2.821922117854188, 0.0, 0.0], [1.410961058927094, 2.4438562415629113, 0.0], [0.0, 0.0, 26.912269282616816]], tags=..., constraint=FixAtoms(indices=[0, 1]), calculator=SinglePointCalculator(...))
name: Pt_111_u1_no01_CO_n1_d3.traj
element: Pt
mirror: 111
adsorbate: CO
unitlength: 1
surfatomnum: 1
area: 6.896371980922386
Energy: -35.77451812
totaladsEnergy: -0.5336636000000041
aveadsEne/suratom: -0.5336636000000041
numberofads: 1
ML: 1.0
density: 0.14500377919960322
aveadsEne/ads: -0.5336636000000041
One post: 5d5ea17abfa5790697e5a40d
571 Pt_111_u1_no02_CO_n1_d3.traj
Atoms(symbols='Pt4', pbc=[True, True, False], cell=[[2.821922117854188, 0.0, 0.0], [1.410961058927094, 2.4438562415629113, 0.0], [0.0, 0.0, 26.912269282616816]], tags=..., constraint=FixAtoms(indices=[0, 1]), calculator=SinglePointCalculator(...))
name: Pt_111_u1_no02_CO_n1_d3.traj
element: Pt
mirror: 111
adsorbate: CO
unitlength: 1
surfatomnum: 1
area: 6.896371980922386
E

name: Pt_111_u2_no09_CO_n2_d3.traj
element: Pt
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 27.585487923689545
Energy: -113.96733623
totaladsEnergy: -2.0761639600000024
aveadsEne/suratom: -0.5190409900000006
numberofads: 2
ML: 0.5
density: 0.07250188959980161
aveadsEne/ads: -1.0380819800000012
One post: 5d5ea17abfa5790697e5a41a
597 Pt_111_u2_no10_CO_n3_d3.traj
Atoms(symbols='Pt16', pbc=[True, True, False], cell=[[5.643844235708376, 0.0, 0.0], [2.821922117854188, 4.8877124831258225, 0.0], [0.0, 0.0, 26.912269282616816]], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7]), calculator=SinglePointCalculator(...))
name: Pt_111_u2_no10_CO_n3_d3.traj
element: Pt
mirror: 111
adsorbate: CO
unitlength: 2
surfatomnum: 4
area: 27.585487923689545
Energy: -128.61702873
totaladsEnergy: -2.240420149999977
aveadsEne/suratom: -0.5601050374999943
numberofads: 3
ML: 0.75
density: 0.10875283439970242
aveadsEne/ads: -0.7468067166666591
One post: 5d5ea17abfa5790697e5a41b
599 Pt_1